In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt

from fit_diagnostics import abs_residuals
from plot_utils import antpos_map, plot_res, plot_res_grouped, plot_res_heatmap
from red_utils import get_bad_ants, find_opt_df, flt_ant_coords, find_zen_file, \
split_rel_results

In [ ]:
%matplotlib inline

In [ ]:
jd_time = 2458098.43869
pol = 'ee'
dist = 'cauchy'
dir_path = '.'

In [ ]:
rel_df = find_opt_df(jd_time, pol, dist, dir_path)

In [ ]:
with open('rel_df.{}.{}.md.pkl'.format(jd_time, pol), 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(rel_df)
df

In [ ]:
test_freq = 600
test_tint = 0

resx = df.loc[(test_freq, test_tint)][5:-2].values.astype(float)

In [ ]:
# check results for a given frequency & time integration
test_freq = 600
test_tint = 0

no_ants = md['no_ants']
no_deg_params = 4
resx = df.loc[(test_freq, test_tint)][5:-2].values.astype(float)
test_gains_comps, test_deg_params, test_vis_comps = np.split(resx, [2*no_ants, \
                                                    2*no_ants+no_deg_params,])

test_gamps = test_gains_comps[::2]
test_gphases = test_gains_comps[1::2]
mean_test_amps = np.mean(test_gamps)
mean_test_phases = np.mean(test_gphases)

print('Gain amplitude average is: {}'.format(mean_test_amps))
print('Gain phase average is: {}'.format(mean_test_phases))

In [ ]:
antpos = flt_ant_coords(jd_time, md['antpos'])

In [ ]:
antpos_map(test_gamps, antpos, 'Gain amplitudes', center=1)
print('Gain amplitude solutions for frequency channel {} and time integration {} are: '\
      '\n{}'.format(test_freq, test_tint, test_gamps))

In [ ]:
antpos_map(test_gphases, antpos, 'Gain phases')
print('Gain phase solutions are:\n{}'.format(test_gphases))

In [ ]:
# success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
plot_res(df, 'nit', clip=True)

In [ ]:
plot_res_grouped(df, 'nit', logy=True)

In [ ]:
plot_res_heatmap(df, 'nit', clip=True)

### Log-likelihood

In [ ]:
plot_res(df, 'fun', clip=True)

In [ ]:
plot_res_grouped(df, 'fun', logy=True)

In [ ]:
plot_res_heatmap(df, 'fun', clip=True)

### Residuals

Look at both the real and imaginary of the the median absolute normalized residual

In [ ]:
df[['med_abs_norm_res_Re', 'med_abs_norm_res_Im']] = df.apply(lambda row: \
pd.Series(abs_residuals(row['norm_residual'])), axis=1)

In [ ]:
plot_res_heatmap(df, value='med_abs_norm_res_Re', clip=True)

In [ ]:
plot_res_heatmap(df, value='med_abs_norm_res_Im', clip=True)

### Stability of gains, visibility solutions and degenerate parameters

#### Gains

In [ ]:
gains_df = df.iloc[:, 5:2*no_ants+5]

gains_df['gamp_mean'] = gains_df.apply(lambda row: np.mean(row[:2*no_ants:2]), axis=1)
gains_df['gphase_mean'] = gains_df.apply(lambda row: np.mean(row[1:2*no_ants:2]), axis=1)

In [ ]:
plot_res(gains_df, 'gamp_mean', ylim=(0.9, 1.1))

In [ ]:
plot_res(gains_df, 'gphase_mean', ylim=(-np.pi, np.pi))

#### Visibility solutions

In [ ]:
no_resid_p = 4 # number of residual parameters in df dataframe
no_min_p = 5 # number of columns in df that are attributes of the SciPy OptimizeResult 
vis_df = df.iloc[:, 2*no_ants+no_min_p+no_deg_params:-no_resid_p]

no_unq_bls = md['no_unq_bls']
vis_df['vamp_mean'] = vis_df.apply(lambda row: np.mean(row[:2*no_unq_bls:2]), axis=1)
vis_df['vphase_mean'] = vis_df.apply(lambda row: np.mean(row[1:2*no_unq_bls:2]), \
                                     axis=1)

In [ ]:
plot_res(vis_df, 'vamp_mean', ylim=(-0.01, 0.01))

In [ ]:
plot_res_heatmap(vis_df, value='vamp_mean', vmax=0.01, vmin=-0.01)

In [ ]:
plot_res(vis_df, 'vphase_mean', ylim=(-np.pi, np.pi))

#### Degenerate parameters

In [ ]:
degp_df = df.iloc[:, 2*no_ants+no_min_p:2*no_ants+no_min_p+no_deg_params]
degp_df.columns = ['overall_amp', 'overall_phase', 'phase_grad_x', 'phase_grad_y']

In [ ]:
plot_res(degp_df, 'overall_amp')

In [ ]:
plot_res_heatmap(degp_df, value='overall_amp', center=1, cmap='bwr', vmin=0.9, vmax=1.1)

In [ ]:
plot_res(degp_df, 'phase_grad_x')

In [ ]:
plot_res_heatmap(degp_df, value='phase_grad_x')

In [ ]:
plot_res(degp_df, 'phase_grad_y')

In [ ]:
plot_res_heatmap(degp_df, value='phase_grad_y', cmap='bwr', center=0)